# Proyecto I: Implementacion de un sistema de Reconocimiento Automático de Habla.

El objetivo de este proyecto es utilizar las herramientas vistas en clase para la implementación de un sistema de reconocimiento automático de habla (ASR-Automatic Speech Recognition). Para lograr este objetivo se realizarán dos implementaciones. La primera de ellas utilizará HMM, y est definida en el **Taller II: Implementing a simple ASR system using HMM**. La segunda Implementación se realizará utilizando redes Neuronales Reurrentes (RNN).

Par al aimplementación del sistema utilizando RNNs, se utilizara la libreria [TensorFlow](https://pypi.org/project/tensorflow/) de Python, especificamente las funciones para la creación de redes neuronales en [keras](https://www.tensorflow.org/guide/keras/sequential_model) y en particular las relacionadas con las redes neuronales recurrentes [(RNN)](https://www.tensorflow.org/guide/keras/rnn). Pueden utilizar las RNN simples, LTSM o las GRU. Sin emabrgo, las que se estudiaron en clase hasta el momento son las RNN simples. Por otro lado no es necesario que usen las RNNs bidirecionales, o las funciones optimizadas par aGPUs.

Para este taller deben seguir los siguientes pasos:

1. Cree una base de datos de entrenamiento, utilizando la segmentación de las palabras en fonemas y los espectrogramas de la señal de voz calculados en el Taller II.
2. Divida los datos entre datos de entrenamiento y validación. Esto lo puede realizar por medio del uso de  la función [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de SciKit-Learn.
3. Construya un aarquitectura para el ASR usando redes neuronales.
4. Evalue el comportamiento del modelo.
5. Ajuste el modelo si lo considera adecuado.
6. Pruebe con los datos de entrenamiento si el modelo produce la secuencia de fonemas indicada.
7. Con el conjunto de palabras de prueba, generado en el taller II, trate de predecir la palbara escrita utilizando el modelo implementado.
8. Discuta sobre el comportamiento del sistema ASR utilziando HMM y RNN. La discusión debe contener al menos la respuest aa las siguientes pregutnas:
    * ¿Cúal modelo e smás facil de entender?
    * ¿Qué modelo funciona mejor? ¿Cúal es la razón para esto?
    * ¿Discuta sobre las ventajas y desventajas del modelo basado en HMM?
    * ¿Discuta sobre las ventajas y desventajas del modelo basado en RNN?
    * ¿Cómo se podria mejorar el sistema desarrollado? ¿Qué hace falta en este sistema ASR?
    * ¿Obtuvó los resultados esperados?
    
Al enviar el proyecto debe incluir los siguientes items:
1. Notebook de Jupyter explicando el desarrollo del proyecto, y con la respuesta a las preguntas realiadas.
2. Archivos de soporte utilizadso, funciones, etc..
3. Grabaciones de las señales de voz utilizadas para entrenar el sistema.
4. Grabaciones de las señales de voz utilizadas para probar el sistema.

**Nota I:** Una guía rápida par ala implementación del modelo de red neuronal utilizando TensorFlow y Python lo pueden encontrar en este [link](https://www.youtube.com/watch?v=BSpXCRTOLJA).

**Nota II:** Recuerde que este proyecto se realiza en grupos de máximo dos personas. También tenga en cuenta que debido a que el taller II hace parte de la evaluación, deben hacerse con el mismo compañero con el que trabajarón ese taller.

**Nota III:** El deadline par ala entrega del proyecto es el **Domingo 28 de Febrero del 2021**.

**Mucha Suerte!!**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
import scipy as sc
from scipy import signal
from scipy.fft import fftshift

import pandas as pd

import librosa
import librosa.display

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import soundfile as sf
import os

from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [25]:
fs = 8000 # Numero de muestras por segundo
nBits = 16 # Numero de bits por muestra del audio
ID = -1
seconds = 5 # Duracion de la grabacion
Nfft = 512
fm = np.arange(0, Nfft / 4) * fs / Nfft

In [26]:
nombres = os.listdir('./utils/sounds/')
nombres

['ahijado.wav',
 'balocesto.wav',
 'espantapajaro.wav',
 'jamon.wav',
 'kiosko.wav',
 'llorar.wav',
 'muchacho.wav',
 'murcielago.wav',
 'sound1.wav',
 'sound10.wav',
 'sound2.wav',
 'sound3.wav',
 'sound4.wav',
 'sound5.wav',
 'sound6.wav',
 'sound7.wav',
 'sound8.wav',
 'sound9.wav',
 'terremoto.wav',
 'zapato.wav']

# Palabras de prueba:
### Para test
1. Tapabocas
2. Teclado
3. Cucaracha
4. Estornudar
5. Coronavirus

6. Vacuna
7. Escoba
8. Recogedor
9. Trapero
10. Sosobra

### Palabras con raiz similar
1. Fechoría
2. Fetiche
3. Satisfecho

4. Zapatería
5. Zapatero

In [27]:
cantidad_fonemas = [7,10,13,5,6,5,6,10,4,8,5,5,5,8,6,7,6,5,8,6]
dicc = {}
for idx,i in enumerate(nombres):
    dicc[i] = cantidad_fonemas[idx]
dicc

{'ahijado.wav': 7,
 'balocesto.wav': 10,
 'espantapajaro.wav': 13,
 'jamon.wav': 5,
 'kiosko.wav': 6,
 'llorar.wav': 5,
 'muchacho.wav': 6,
 'murcielago.wav': 10,
 'sound1.wav': 4,
 'sound10.wav': 8,
 'sound2.wav': 5,
 'sound3.wav': 5,
 'sound4.wav': 5,
 'sound5.wav': 8,
 'sound6.wav': 6,
 'sound7.wav': 7,
 'sound8.wav': 6,
 'sound9.wav': 5,
 'terremoto.wav': 8,
 'zapato.wav': 6}

In [28]:
def getAudios(nombres):
    audios = []
    for i in nombres:
        y_aux, fs_aux = sf.read('./utils/sounds/' + i)
        if len(y_aux.shape) > 1:
            y_aux = y_aux[:,0]
            y = ((1/np.std(y_aux))*y_aux).reshape(len(y_aux))
        else:
            y = ((1/np.std(y_aux))*y_aux).reshape(len(y_aux))
        audios.append(y)
    return audios

In [29]:
def getMelSpectogram(audios,fs,Nfft):
    spectograms = []
    for i in audios:
        Sm = librosa.feature.melspectrogram(y=i, sr=fs, n_fft=Nfft, n_mels = 39)
        spectograms.append(Sm)
    return spectograms

In [30]:
audios = getAudios(nombres)

In [31]:
Spectograms = getMelSpectogram(audios,fs,Nfft)

In [32]:
palabras_codificadas = [[1,9,10,11,1,5,18], [2,1,13,18,16,3,6,22,23,18],[6,22,19,1,16,23,1,19,1,11,1,20,18],
                        [11,1,15,18,16],[12,10,18,22,12,18], [14,18,20,1,20],[15,24,4,1,4,18],[15,24,20,3,10,6,13,1,8,18],
                        [7,6,4,1], [2,10,14,6,23,6,20,1],[13,10,2,20,18],[13,1,19,10,3],[12,18,21,6,20],[23,6,13,6,7,18,16,18],
                        [19,24,6,20,23,1],[19,1,16,23,1,14,1],[11,10,15,6,16,1],[21,6,8,13,1],[23,6,21,6,15,18,23,18],[3,1,19,1,23,18]]
for p in palabras_codificadas:
    p.insert(0,0)
    p.append(0)

In [33]:
dicc_countFon = {}
for i in range(0,27):
    ocurrencia = 0
    for j in palabras_codificadas:
        ocurrencia += j.count(i)
    dicc_countFon[i] = ocurrencia

In [34]:
def getOneCode(palabras_codificadas):
    palabras_onecode = []
    for palabra in palabras_codificadas:
        matriz = np.zeros((24,len(palabra)))
        for i in range(len(palabra)):
            if palabra[i] > 17 and palabra[i] <= 24:
                matriz[palabra[i] - 1,i] = 1
            else:
                matriz[palabra[i],i] = 1
        palabras_onecode.append(matriz)
    return palabras_onecode

In [35]:
matrices_1_0 = getOneCode(palabras_codificadas)

# Pregunta

Cual es la interpretacion de realizar el split de esta forma

In [36]:
from copy import deepcopy

In [14]:
# Cada matriz de entrenamiento tiene 39 filas y n columnas
X_train, X_test, y_train, y_test = train_test_split(Spectograms, matrices_1_0, test_size=0.3)

In [15]:
[m.shape for m in y_train]

[(24, 8),
 (24, 9),
 (24, 10),
 (24, 7),
 (24, 9),
 (24, 10),
 (24, 12),
 (24, 8),
 (24, 8),
 (24, 10),
 (24, 7),
 (24, 15),
 (24, 8),
 (24, 6)]

# Modelo
## Modificación de las dimensiones de X 

In [39]:
# Agregando matriz de espectrograma a tensor
# Se crea un tensor de (p,q,r) con p: numero de espectrogramas, q: numero de filas, r: numero de columnas
# r se fija en el valor maximo de numero de columnas entre todos los espectrogramas para que todos los 
# espectrogramas tengan la misma dimension en el tensor


# cada espectrograma de X tiene dimension (39,max_cols) siendo max_cols
# el numero maximo de columnas entre todos los espectrogramas

# Para datos de entrenamiento 
max_dim_tr = max([X_train[i].shape[1] for i in range(len(X_train))])
X_tr_new = np.zeros((len(X_train),max_dim_tr,39))
for i in range(len(X_tr_new)):
    X_tr_new[i,0:X_train[i].shape[1],:] = X_train[i].T
    
# Para datos de prueba
max_dim_test = max([X_test[i].shape[1] for i in range(len(X_test))])
X_test_new = np.zeros((len(X_test),max_dim_test,39))
for i in range(len(X_test_new)):
    X_test_new[i,0:X_test[i].shape[1],:] = X_test[i].T

In [53]:
print(X_tr_new.shape)
#[m.shape for m in y_tr_new] # (15, 24)

(14, 242, 39)


[(15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24),
 (15, 24)]

## Modificacion de dimensiones de y

In [54]:
# Se agrega columnas de silencio a las matrices de y tal que cada matriz de y_train y y_test
# (por separado) tiene dimension (24,max_n_train) o (24,max_n_test) 

# Para outputs de entrenamiento 
max_dim_tr = max([y_train[i].shape[1] for i in range(len(y_train))])
max_dim_test = max([y_test[i].shape[1] for i in range(len(y_test))])
MAX = max([max_dim_tr,max_dim_test])

y_train_new = []
new_col = np.array([1]+23*[0]).reshape([24,1])
for i in range(len(y_train)):
    mat_copy = deepcopy(y_train[i])
    add_left_flag = True
    while mat_copy.shape[1] < MAX:
        if add_left_flag:
            mat_copy = np.concatenate([new_col,mat_copy],axis=1)
        else:
            mat_copy = np.concatenate([mat_copy,new_col],axis=1)
    y_train_new.append(mat_copy)

# Para datos de prueba
y_test_new = []
new_col = np.array([1]+23*[0]).reshape([24,1])
for i in range(len(y_test)):
    mat_copy = deepcopy(y_test[i])
    add_left_flag = True
    while mat_copy.shape[1] < MAX:
        if add_left_flag:
            mat_copy = np.concatenate([new_col,mat_copy],axis=1)
        else:
            mat_copy = np.concatenate([mat_copy,new_col],axis=1)
    y_test_new.append(mat_copy)

In [55]:
# Transformando outputs de entrenamiento y de salida en tensores

# Para y_train
y_tr_new = np.zeros((len(y_train_new), y_train_new[0].shape[1], y_train_new[0].shape[0]))
for i in range(len(y_train_new)):
    y_tr_new[i,:,:] = y_train_new[i].T
    
# Para y_test
y_tes_new = np.zeros((len(y_test_new), y_test_new[0].shape[1], y_test_new[0].shape[0]))
for i in range(len(y_test_new)):
    y_tes_new[i,:,:] = y_test_new[i].T

In [56]:
print(y_tr_new.shape)
print(y_tes_new.shape)

(14, 15, 24)
(6, 15, 24)


## Intento 1 de RNN

In [23]:
model = Sequential()
# Numero arbitrario?
model.add(LSTM(128, input_shape=(X_tr_new.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.001))

#model.add(LSTM(128, activation='relu'))
#model.add(Dropout(0.1))

model.add(LSTM(24, input_shape=(X_tr_new.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

# Capa de salida: numero de clases de salida: numero de fonemas utilizados
#n_out = 24 # fonemas que aparecen en palabras incluyebndo el silencio
#model.add(Dense(n_out, activation='softmax'))


## Intento 2 de RNN

In [24]:
model = Sequential()
model.add(LSTM(12, input_shape=(X_tr_new.shape[1:]), return_sequences=True))
model.add(tf.keras.layers.TimeDistributed(Dense(24)))

#model.add(tf.keras.layers.Lambda(lambda x: x[:, -24:, :])) #Select last N from output

## Intento 3 de RNN

In [65]:
model = Sequential()  
model.add(LSTM(24,input_dim=39, return_sequences=False))
model.add(Dense(24,tf.keras.layers.Activation('relu')))
#
model.add(tf.keras.layers.RepeatVector(y_train_new[0].shape[1])) # columnas en y (param.2)
model.add(LSTM(24, return_sequences=True))  
# Capa de salida
model.add(tf.keras.layers.TimeDistributed(Dense(24)))
model.add(tf.keras.layers.Activation('linear'))
#opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-8) # rmsprop
model.compile(loss='mean_squared_error', optimizer="rmsprop", metrics=['accuracy'])  

In [46]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-8)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

In [ ]:
print(X_tr_new.shape)
print(y_tr_new.shape)

In [ ]:
print(X_test_new.shape)
print(y_tes_new.shape)

In [66]:
model.fit(X_tr_new,
          y_tr_new,
          epochs=50,
          validation_data=(X_test_new, y_tes_new))

Epoch 1/50
1/1 [==============================] - 13s 13s/step - loss: 0.0422 - accuracy: 0.1190 - val_loss: 0.0402 - val_accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 249ms/step - loss: 0.0403 - accuracy: 0.4571 - val_loss: 0.0389 - val_accuracy: 0.6000
Epoch 3/50
1/1 [==============================] - 0s 250ms/step - loss: 0.0392 - accuracy: 0.5238 - val_loss: 0.0379 - val_accuracy: 0.6000
Epoch 4/50
1/1 [==============================] - 0s 265ms/step - loss: 0.0383 - accuracy: 0.5286 - val_loss: 0.0370 - val_accuracy: 0.6000
Epoch 5/50
1/1 [==============================] - 0s 250ms/step - loss: 0.0376 - accuracy: 0.5286 - val_loss: 0.0362 - val_accuracy: 0.6000
Epoch 6/50
1/1 [==============================] - 0s 248ms/step - loss: 0.0369 - accuracy: 0.5286 - val_loss: 0.0355 - val_accuracy: 0.6000
Epoch 7/50
1/1 [==============================] - 0s 253ms/step - loss: 0.0362 - accuracy: 0.5286 - val_loss: 0.0348 - val_accuracy: 0.6000
Epoch 8/50
1/1 [=====

# Sugerencias
- Grabar varias veces las mismas palabras.
- Preguntar a Juan Miguel